Feed forward modell jeg bruker til å teste sammenhenger mellom sentiment, finansdata og markedsutvikling.
inputet til modellen må være en csv fil med format str([[data-input],[resultat]]).
Dataen må være normalisert på forhånd

In [ ]:
import pandas as pd
import numpy as np
file = pd.read_csv('input file')
x = file.X
y = file.Y

x_list = []
y_list = []
number_of_tags = len(y[0].strip('][[]').split(','))

for row,row_y in zip(x,y):
    x_point = row.strip('][[]').split(',')
    x_list.append([float(i) for i in x_point])
    
    y_point = row_y.strip('][[]').split(',')
    y_list.append(np.argmax([int(i) for i in y_point]))
print(len(x_list), len(x_list[0]))
print(len(y_list),number_of_tags)

For å få bedre resultat fra treningen må dataen stokkes, det gjøres i følgende blokk.

In [ ]:
import random
indexes = [i for i in range(0,len(x_list))]
random.shuffle(indexes)
y_list_r = []
x_list_r = []
for index in indexes:
    y_list_r.append(y_list[index])
    x_list_r.append(x_list[index])

Dette er dataloader steget. Hvis du ikke har cuda tilgjengelig vil det bli valgt cpu, merk at dette vil gjøre treningen mye tregere

In [ ]:
import torch

from torch.utils.data import TensorDataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


x_tensor = torch.Tensor(x_list_r)
y_tensor = torch.Tensor(y_list_r)



train_dataset = TensorDataset(x_tensor[0:int(len(x_list)*0.8)], y_tensor[0:int(len(x_list)*0.8)])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = TensorDataset(x_tensor[int(len(x_list)*0.8):], y_tensor[int(len(x_list)*0.8):])
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

For at resultatet ikke skal bli påviket av fordelingen av dataen må den vektes, det gjøres på følgende måte.

In [ ]:
d = {0:0,1:0}
for i in y_tensor[0:int(len(x_list)*0.8)]:
    d[int(i.item())] += 1
weights = [max(d.values())/d[i] for i in d]

Modellen som er brukt er en sequntail linear modell, med ReLU aktiveringsfunksjon

In [ ]:
import torch.nn as nn
#første input er antall kolloner i dataen
#siste input er antall klasser i datan
model = nn.Sequential(
          nn.Linear(len(x_list[0]), 300),
          nn.ReLU(),
          nn.Linear(300, 600),
          nn.ReLU(),
          nn.Linear(600, 32),
          nn.ReLU(),
          nn.Linear(32, number_of_tags)
          )
model.to(device)

Neste steg er trene modellen, jeg har brukt weight_decay=0.0005, men weight_decay er varierende og noe man optimalisere på nytt for hvert datasett

In [ ]:
import torch.optim as optim

loss_weights = torch.Tensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight=loss_weights)
optimizer = optim.Adam(model.parameters(), weight_decay=0.0005)

for epoch in range(2000):  # loop over the dataset multiple times

    correct_train = 0
    total_train = 0
    
    running_loss_train = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(inputs)
        
        _, predicted = torch.max(outputs.data, 1)
        #predicted = predicted.to(device)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
        
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss_train += loss.item()
    
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in test_dataloader:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            #predicted = predicted.to(device)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    if (epoch % 10 == 0):
        print('[%d] Accuracy (train): %d %%, Accuracy (test): %d %%, loss: %.3f' % \
              (epoch, 100 * correct_train / total_train, 100 * correct / total, running_loss_train\
               / len(train_dataloader)))

print('Finished Training')

For å teste modellen kan man fylle inn test_data_inpu

In [ ]:
test_data_input = 

from scipy.special import softmax
test = torch.Tensor(test_data_input).to(device)

output = model(test).tolist()

result = softmax(output)
print(result)